In [1]:
import requests 
import pickle
import json
import psycopg2
import pandas as pd

In [2]:
feature_store_id = "cdc74d4c"

In [3]:
# Connect to your postgres DB
conn = psycopg2.connect(
    dbname="olap",
    user="dsp",
    password="dsppassword",
    host="localhost",
    port="5432")

# Open a cursor to perform database operations
cur = conn.cursor()

# Execute a query
cur.execute("SELECT * FROM ctr limit 1;")

# Retrieve query results
records = cur.fetchall()

In [4]:
ctr_data = pd.DataFrame(
    records, 
    columns=[
        "ad_id",
        "status",
        "bidding_cpc",
        "advertiser",
        "banner_style",
        "category",
        "height",
        "width",
        "item_price",
        "layout_style",
        "hist_ctr",
        "hist_cvr",
        "was_click"
    ]
)

In [5]:
# assume this is how the data look on the client side
inference_client_input = ctr_data.iloc[0].to_dict()
print(inference_client_input)

{'ad_id': 1228, 'status': False, 'bidding_cpc': 1, 'advertiser': 'QQ', 'banner_style': 'VII', 'category': 'Pullover', 'height': 27.341917, 'width': 617.5163, 'item_price': 2896.437, 'layout_style': 'MP', 'hist_ctr': 0.00036614624, 'hist_cvr': 6.790803e-05, 'was_click': False}


In [6]:
# load the model
with open('../data/model-default/model.pickle', 'rb') as file:
    model = pickle.load(file)

/Users/k/Library/Caches/pypoetry/virtualenvs/research-3QkaudzG-py3.9/lib/python3.9/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator LogisticRegression from version 1.3.0 when using version 1.1.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [7]:
# this is the feature store that create from modeling_with_feature_store.ipynb
# the feature store is able to help us do the feature engineering part
response = requests.get(
    "http://localhost:8000/feature_store/{feature_store_id}".format(feature_store_id=feature_store_id))
print(json.dumps(response.json(),indent=2))

{
  "feature_store_id": [
    "cdc74d4c"
  ],
  "feature_store_name": [
    "mini-demand-side-platform"
  ],
  "description": [
    "choice top 10 feature from correlation"
  ],
  "offline_table_name": [
    "top_10_features"
  ]
}


In [8]:
# check available features
response = requests.get(
    "http://localhost:8000/feature_store/{feature_store_id}/feature".format(feature_store_id=feature_store_id))
features = response.json()
print(features)

{'feature_id': ['5dbfb549', 'ea2f676f', '0ad63312', 'a2480212', '5aae0b22', '65934929', '2658049e', '717456fc', '1d34889f', '915a0c3c'], 'feature_name': ['layout_style_AB', 'layout_style_RU', 'layout_style_GY', 'layout_style_MR', 'layout_style_BK', 'layout_style_BX', 'layout_style_RZ', 'layout_style_TY', 'category_Shirt', 'layout_style_DX'], 'source_table_name': ['ctr', 'ctr', 'ctr', 'ctr', 'ctr', 'ctr', 'ctr', 'ctr', 'ctr', 'ctr'], 'source_column_name': ['layout_style', 'layout_style', 'layout_style', 'layout_style', 'layout_style', 'layout_style', 'layout_style', 'layout_style', 'category', 'layout_style'], 'feature_function_type': ['string_mapping', 'string_mapping', 'string_mapping', 'string_mapping', 'string_mapping', 'string_mapping', 'string_mapping', 'string_mapping', 'string_mapping', 'string_mapping']}


In [9]:
def convert_client_data_for_online_prediction(data, features):
    res = {"inputs":[]}
    for i in range(len(features["source_column_name"])):
        res["inputs"].append(data[features["source_column_name"][i]])
    res["feature_ids"] = features["feature_id"]
    res["feature_store_function_types"] = features["feature_function_type"]
    return res

In [10]:
get_online_features_input = convert_client_data_for_online_prediction(data=inference_client_input,features=features)
print(get_online_features_input)

{'inputs': ['MP', 'MP', 'MP', 'MP', 'MP', 'MP', 'MP', 'MP', 'Pullover', 'MP'], 'feature_ids': ['5dbfb549', 'ea2f676f', '0ad63312', 'a2480212', '5aae0b22', '65934929', '2658049e', '717456fc', '1d34889f', '915a0c3c'], 'feature_store_function_types': ['string_mapping', 'string_mapping', 'string_mapping', 'string_mapping', 'string_mapping', 'string_mapping', 'string_mapping', 'string_mapping', 'string_mapping', 'string_mapping']}


In [11]:
# Get the formated data
response = requests.post(
    "http://localhost:8000/online_features",json=get_online_features_input)
online_features = response.json()
print(online_features)

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


In [12]:
model.predict_proba([online_features])

array([[0.99861459, 0.00138541]])